In [1]:
%%javascript
$.getScript('http://asimjalis.github.io/ipyn-ext/js/ipyn-present.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">MLlib</h1>
<div id="toc"></div>

## MLlib History

MLlib is a Spark subproject providing machine learning primitives

Initial contribution from AMPLab, UC Berkeley

Shipped with Spark since Sept 2013

## Example: Text Classification
1. Start with RDD of strings
2. Run _feature extraction_ to convert text into numerical features.
3. Call classification algorithm on RDD of vectors
4. Evaluate model on test dataset using evaluation functions.

_N.B._ You do not _need_ to use MLlib to do machine learning on Spark. Depending on the nature of the problem, it may make more sense to use a single-node machine learning library (_i.e._ `scikit-learn`) on multiple smaller datasets for such embarassingly parallel problems as grid search, for example (though MLlib also has grid-search built in, see `ParamGridBuilder` and `CrossValidator` below)

## Data Types
* `Vector` - can be dense or sparse; see `mllib.linalg.Vectors`
* `LabeledPoint` - labeled data point for supervised learning algorithms; see `mllib.regression`
* `Rating` - rating of product by a user; see `mllib.recommendation`
* *Various* `Model` *classes* - typically has a `predict()` method (similar to `scikit-learn` API)

### Pop Quiz
<details><summary>
Q: Which data type would you use for random forests?
</summary>
A: `LabeledPoint`. Random forests is a supervised learning algorithm. It requires labels in order to classify.
</details>

## MLlib: Available algorithms

**feature extraction:** HashingTF, IDF, StandardScaler, Normalizer, Word2Vec

**statistics:** mean, stdev, sum, corr, chiSqTest

**classification:** logistic regression, linear SVM," naïve Bayes, least squares, classification tree  

**regression:** generalized linear models (GLMs), regression tree

**collaborative filtering:** alternating least squares (ALS), non-negative matrix factorization (NMF)

**clustering:** k-means||

**decomposition:** SVD, PCA

**optimization:** stochastic gradient descent, L-BFGS

### Feature Extraction

### Statistics

**`Statistics.colStats(`*rdd*`)`:** returns min, max, mean, and variance  

#### Pop Quiz:
<p><details><summary>Q: What common statistics are missing from this list? Why?</summary>
A: Median and other quantiles are missing.  
Why? Because they are non-associative.
</details></p>

**`Statistics.corr(`*rdd*`, `*method*`)`:** Computes correlation matrix between columns in RDD of vectors, using either the Pearson or Spearman correlation  

**`Statistics.corr(`*rdd1*`, `*rdd2*`, `*method*`)`:** Computes the correlation between two RDDs (*method* must be one of `pearson` and `spearman`).

#### Pop Quiz:
<p><details><summary>Q: What's the difference between Pearson and Spearman correlation?</summary>
A: Pearson's $r$ is parametric. Spearman's $\rho$ is non-parametric.  
Q: What's the difference between parametric and non-parmetric?
</details></p>

In addition, RDDs support `sample()` and `sampleByKey()` to build simple and stratified samples of data.

_N.B._ This can be very useful for bootstrapping.

Classification and Regression
-----------------------------

_N.B._ For binary classification, MLlib expects the labels $0$ and $1$. In some texts, $–1$ and $1$ are used instead, but this will lead to incorrect results. For multiclass classification, MLlib expects labels from $0$ to $C–1$, where $C$ is the number of classes.

-----------------------------

Almost all machine learning objectives are optimized using this update
$$w\leftarrow w-\alpha\cdot\sum_{i=1}^ng(w;x_i,y_i)$$
$w$ is a vector of dimension $d$  
we’re trying to find the best $w$ via optimization

## Scaling
1. Data size
2. Number of models
3. Model size

## Logistic Regression
Goal: find best line separating two sets of points

$$w\leftarrow w-\alpha\cdot\sum_{i=1}^ng(w;x_i,y_i)$$

```python
points = sc.textFile(...).mapPartitions(readPointBatch).cache()

w = 2 * np.random.ranf(size=D) - 1

def gradient(matrix, w):
    Y = matrix[:, 0]    # point labels (first column of input file): -1 or 1
    X = matrix[:, 1:]   # point coordinates
    # For each point (x, y), compute gradient function, then sum these up
    return ((1.0 / (1.0 + np.exp(-Y * X.dot(w))) - 1.0) * Y * X.T).sum(1)

for i in range(iterations):
    w -= points.map(lambda m: gradient(m, w)).reduce(add)
```

## Separable Updates

Can be generalized for

* Unconstrained optimization 

* Smooth or non-smooth

* LBFGS, Conjugate Gradient, Accelerated Gradient methods, ...

## Logistic Regression Results
![](http://a3ab771892fd198a96736e50.javacodegeeks.netdna-cdn.com/wp-content/uploads/2015/05/hadoop_spark_comparison.png)

## Lots of little models

Is embarrassingly parallel

Most of the work should be handled by data flow paradigm

ML pipelines does this

## Hyper-parameter Tuning
```python
# Build a parameter grid.
paramGrid = ParamGridBuilder() \
                .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()

# Set up cross-validation.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3) 
                          
# Fit a model with cross-validation.
cvModel = crossval.fit(training)
```
see http://spark.apache.org/docs/latest/ml-guide.html  
and https://github.com/apache/spark/blob/master/examples/src/main/python/ml/cross_validator.py#L69

---

## Optimization

At least two large classes of optimization problems humans can solve:  

* Convex Programs

* Spectral Problems

---

# Matrix Completion with ALS

## Collaborative Filtering

Goal: predict users’ movie ratings based on past ratings of other movies

$$
R = \left( \begin{array}{ccccccc}
1 & ? & ? & 4 & 5 & ? & 3 \\
? & ? & 3 & 5 & ? & ? & 3 \\
5 & ? & 5 & ? & ? & ? & 1 \\
4 & ? & ? & ? & ? & 2 & ?\end{array} \right)
$$

Don’t mistake this with SVD.

Both are matrix factorizations, however SVD cannot handle missing entries.

![Matrix Factorization](https://databricks-training.s3.amazonaws.com/img/matrix_factorization.png)

## Alternating Least Squares  
![ALS](https://databricks.com/wp-content/uploads/2014/07/als-illustration.png)  
1. Start with random $U_1$, $V_1$
2. Solve for $U_2$ to minimize $||R – U_2V_1^T||$ 
3. Solve for $V_2$ to minimize $||R – U_2V_2^T||$ 
4. Repeat until convergence

## ALS on Spark

Cache 2 copies of $R$ in memory, one partitioned by rows and one by columns 

Keep $U~\&~V$ partitioned in corresponding way 

Operate on blocks to lower communication


---
Lab
---
[Movie Recommendation with MLlib](lab.ipynb)

<!-- Advanced:

# Distributing Matrix Computations

## Distributing Matrices

How to distribute a matrix across machines?  

* By Entries (CoordinateMatrix)

* By Rows (RowMatrix)

* By Blocks (BlockMatrix)

All of Linear Algebra to be rebuilt using these partitioning schemes

Even the simplest operations require thinking about communication e.g. multiplication


How many different matrix multiplies needed?

* At least one per pair of {Coordinate, Row, Block, LocalDense, LocalSparse} = 10

* More because multiplies not commutative

# Singular Value Decomposition on Spark

## Singular Value Decomposition
![SVD](http://langvillea.people.cofc.edu/DISSECTION-LAB/Emmie'sLSI-SVDModule/svddiagram.gif)

## Singular Value Decomposition

Two cases:

* Tall and Skinny

* Short and Fat (not really) 

* Roughly Square

SVD method on RowMatrix takes care of which one to call.

*(If I want the SVD of a fat matrix, I can do it by taking the transpose, finding the SVD of this skinny matrix, and then swapping the "U"s and "V"s.)*

## Tall and Skinny SVD

* Given $m \times n$ matrix $A$, with $m \gg n$
* We compute $A^TA$
* $A^TA$ is $n \times n$, considerably smaller than $A$
* $A^TA$ is dense
* Holds dot products between all pairs of columns of $A$.
$$A=U\Sigma V^T \quad A^TA=V\Sigma^2V^T$$

## Tall and Skinny SVD  

* $A^TA=V\Sigma^2V^T$ Gets us V and the singular values  

* $A=U\Sigma V^T$ Gets us U by one matrix multiplication

## Square SVD

ARPACK: Very mature Fortran77 package for computing eigenvalue decompositions"

JNI interface available via netlib-java

Distributed using Spark – how?

## Square SVD via ARPACK

Only interfaces with distributed matrix via matrix-vector multiplies

$$ K_n=[b\;Ab\;A^2b\;\cdots\;A^{n-1}b] $$

The result of matrix-vector multiply is small. 

The multiplication can be distributed.

## Square SVD

|￼ Matrix size    | Number of<br>nonzeros | Time per<br>iteration (s) | Total<br>time (s) |
|:-------------------:| -------------:|:---:|:-:
| 23,000,000 x 38,000 |    51,000,000 | 0.2 | 10
| 63,000,000 x 49,000 |   440,000,000 | 1   | 50
| 94,000,000 x  4,000 | 1,600,000,000 | 0.5 | 50

With 68 executors and 8GB memory in each, looking for the top 5 singular vectors

# Communication-Efficient $A^TA$
### All pairs similarity on Spark (DIMSUM)

## All pairs Similarity

All pairs of cosine scores between n vectors

* Don’t want to brute force (n choose 2) m

* Essentially computes $A^TA$

Compute via [DIMSUM](http://arxiv.org/abs/1304.1467)

* Dimension Independent Similarity Computation using MapReduce

## Intuition

Sample columns that have many non-zeros with lower probability.

On the flip side, columns that have fewer non-zeros are sampled with higher probability.

Results provably correct and independent of larger dimension, m.

## Spark implementation
```scala
// Load and parse the data file.
val rows = sc.textFile(filename).map { line => 
    var values = line.split(' ').map(_.toDouble)
    Vectors.dense(values)
}
val mat = new RowMatrix(rows)

// Compute similar columns perfectly, with brute force.
val simsPerfect = mat.columnSimilarities()

// Compute similar columns with estimation using DIMSUM
val simsEstimate = mat.columnSimilarities(threshold)
```
-->